Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/how-to-use-azureml/explain-model/azure-integration/remote-explanation/explain-model-on-amlcompute.png)

## Setup

In [23]:
# Check core SDK version number
import azureml.core

print("SDK version:", azureml.core.VERSION)

SDK version: 1.0.69


## Initialize a Workspace

Connect to the workspace

In [24]:
from azureml.core import Workspace, Dataset

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep='\n')

ignite
ignite
eastus2
15ae9cb6-95c1-483d-a0e3-b1a1a3b06324


In [25]:
model = ws.models['attrition_python']

### To be safe, let's create an environment to match the current one on this VM

In [32]:
from azureml.core import Environment

env = Environment.from_conda_specification(name = "ScikitEnvironment", file_path='scikit.yaml')

env.register(ws)

{
    "name": "ScikitEnvironment",
    "version": "4",
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "python": {
        "userManagedDependencies": false,
        "interpreterPath": "python",
        "condaDependenciesFile": null,
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "dependencies": [
                "python=3.6.2",
                {
                    "pip": [
                        "azureml-defaults",
                        "scikit-learn==0.20.3",
                        "numpy",
                        "pandas",
                        "sklearn-pandas",
                        "inference-schema[numpy-support]",
                        "joblib"
                    ]
                }
            ],
            "name": "azureml_680cc38ac27203ae288d78d93091b187"
        }
    },
    "docker": {
        "enabled": false,
        "baseImage": "mcr.microsoft.com/azureml/base:intelmpi2018.3-ubuntu16.04",
 

In [33]:
# if already created, just retrieve it
env = ws.environments['ScikitEnvironment']

### Create an Inference Config for our Model

In [34]:
from azureml.core.model import InferenceConfig

inference_config = InferenceConfig(entry_script="score.py", environment=env)

In [35]:
from azureml.core.webservice import AciWebservice, Webservice
from azureml.exceptions import WebserviceException
from azureml.core import Model

deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)
aci_service_name = 'attritionpython'

try:
    # if you want to get existing service below is the command
    # since aci name needs to be unique in subscription deleting existing aci if any
    # we use aci_service_name to create azure aci
    service = Webservice(ws, name=aci_service_name)
    if service:
        service.delete()
except WebserviceException as e:
    print()

service = Model.deploy(ws, aci_service_name, [model], inference_config, deployment_config)



In [36]:
service.wait_for_deployment(True)
print(service.state)

Running............................................................................
SucceededACI service creation operation finished, operation "Succeeded"
Healthy


In [37]:
# if there are any issues whatsoever, there first place to look is here
print(service.get_logs())

2019-10-29T23:44:39,330219679+00:00 - rsyslog/run 
2019-10-29T23:44:39,336532144+00:00 - iot-server/run 
2019-10-29T23:44:39,350368886+00:00 - gunicorn/run 
2019-10-29T23:44:39,369514884+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_680cc38ac27203ae288d78d93091b187/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_680cc38ac27203ae288d78d93091b187/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_680cc38ac27203ae288d78d93091b187/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_680cc38ac27203ae288d78d93091b187/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_680cc38ac27203ae288d78d93091b187/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
EdgeHubC

In [40]:
import pandas as pd
df = pd.read_csv('../data/IBM-Employee-Attrition.csv')

In [44]:
service.run(df.iloc[0:5].to_json())

[[0.228666986281843, 0.771333013718157],
 [0.9853813788998493, 0.014618621100150676],
 [0.4012630320312107, 0.5987369679687893],
 [0.8751925760453446, 0.12480742395465541],
 [0.6940584864926806, 0.3059415135073193]]

## Create An Experiment

**Experiment** is a logical container in an Azure ML Workspace. It hosts run records which can include run metrics and output artifacts from your experiments.

In [ ]:
from azureml.core import Experiment
experiment_name = 'explainer-remote-run-on-amlcompute'
experiment = Experiment(workspace=ws, name=experiment_name)

## Introduction to AmlCompute

Azure Machine Learning Compute is managed compute infrastructure that allows the user to easily create single to multi-node compute of the appropriate VM Family. It is created **within your workspace region** and is a resource that can be used by other users in your workspace. It autoscales by default to the max_nodes, when a job is submitted, and executes in a containerized environment packaging the dependencies as specified by the user. 

Since it is managed compute, job scheduling and cluster management are handled internally by Azure Machine Learning service. 

For more information on Azure Machine Learning Compute, please read [this article](https://docs.microsoft.com/azure/machine-learning/service/how-to-set-up-training-targets#amlcompute)

**Note**: As with other Azure services, there are limits on certain resources (for eg. AmlCompute quota) associated with the Azure Machine Learning service. Please read [this article](https://docs.microsoft.com/azure/machine-learning/service/how-to-manage-quotas) on the default limits and how to request more quota.



## Submit an AmlCompute run in a few different ways

First lets check which VM families are available in your region. Azure is a regional service and some specialized SKUs (especially GPUs) are only available in certain regions. Since AmlCompute is created in the region of your workspace, we will use the supported_vms () function to see if the VM family we want to use ('STANDARD_D2_V2') is supported.

You can also pass a different region to check availability and then re-create your workspace in that region through the [configuration notebook](../../../configuration.ipynb)

### Create project directory

Create a directory that will contain all the necessary code from your local machine that you will need access to on the remote resource. This includes the training script, and any additional files your training script depends on

In [ ]:
import os
import shutil

project_folder = './explainer-remote-run-on-amlcompute'
os.makedirs(project_folder, exist_ok=True)
shutil.copy('train_explain.py', project_folder)

Note: if you need to cancel a run, you can follow [these instructions](https://aka.ms/aml-docs-cancel-run).

### Fetch or create the compute target 

We are going to use the compute target you had created before (make sure you provide the same name here in the variable `cpu_cluster_name`. 

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
cpu_cluster_name = "cpu"

cpu_cluster = ws.compute_targets[cpu_cluster_name]

The above code will throw an error if you didn't set the cluster up in the first part of the workshop. In that case, you can provision a cluster here by executing the next cell.

Note: By default a cluster autoscales from 0 nodes and provisions dedicated VMs to run your job in a container. This is useful when you want to continously re-use the same target, debug it between jobs or simply share the resource with other users of your workspace.

* `vm_size`: VM family of the nodes provisioned by AmlCompute. Simply choose from the supported_vmsizes() above
* `max_nodes`: Maximum nodes to autoscale to while running a job on AmlCompute

In [ ]:
## if you didn't create the cluster in the first part of the workshop, this code will create it
compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                       max_nodes=10, 
                                                       idle_seconds_before_scaledown=7200)
cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

### Configure & Run

In [ ]:
from azureml.train.estimator import Estimator

pip_packages = [
    'azureml-defaults', 'azureml-contrib-interpret', 'azureml-core', 'azureml-telemetry',
    'azureml-interpret', 'sklearn-pandas', 'azureml-dataprep', 'joblib'
]

estimator = Estimator(source_directory=project_folder, 
                      compute_target=cpu_cluster,
                      entry_script='train_explain.py',
                      pip_packages=pip_packages,
                      conda_packages=['scikit-learn'],
                      inputs=[ws.datasets['IBM-Employee-Attrition'].as_named_input('attrition')])

run = experiment.submit(estimator)
run

In [ ]:
from azureml.widgets import RunDetails
RunDetails(run).show()

![](img/aml-run.png)

## Additional operations to perform on AmlCompute

You can perform more operations on AmlCompute such as updating the node counts or deleting the compute. 

In [ ]:
# Get_status () gets the latest status of the AmlCompute target
cpu_cluster.get_status().serialize()


In [ ]:
# Update () takes in the min_nodes, max_nodes and idle_seconds_before_scaledown and updates the AmlCompute target
# cpu_cluster.update(min_nodes=1)
# cpu_cluster.update(max_nodes=10)
cpu_cluster.update(idle_seconds_before_scaledown=7200)
# cpu_cluster.update(min_nodes=2, max_nodes=4, idle_seconds_before_scaledown=600)

In [ ]:
# Delete () is used to deprovision and delete the AmlCompute target. Useful if you want to re-use the compute name 
# 'cpu-cluster' in this case but use a different VM family for instance.

# cpu_cluster.delete()

In [ ]:
run.wait_for_completion(show_output=True)

## Download 
### 1. Download model explanation data.

In [ ]:
from azureml.contrib.interpret.explanation.explanation_client import ExplanationClient

# Get model explanation data
client = ExplanationClient.from_run(run)
global_explanation = client.download_model_explanation()


In [ ]:
# Get the top k (e.g., 4) most important features with their importance values
global_explanation_topk = client.download_model_explanation(top_k=4)
global_importance_values = global_explanation_topk.get_ranked_global_values()
global_importance_names = global_explanation_topk.get_ranked_global_names()

In [ ]:
print('global importance values: {}'.format(global_importance_values))
print('global importance names: {}'.format(global_importance_names))

### 2. Download model and test set files

In [ ]:
# retrieve model for visualization and deployment
from azureml.core.model import Model
import joblib

run.download_file('original_model.pkl')
original_model = joblib.load('original_model.pkl')

run.download_file('x_test_ibm.pkl')
x_test = joblib.load('x_test_ibm.pkl')

## Visualize
Load the visualization dashboard

In [ ]:
from azureml.contrib.interpret.visualize import ExplanationDashboard

In [ ]:
ExplanationDashboard(global_explanation, original_model, x_test)